Create Bucket in Google Cloud

In [1]:
!gcloud auth list

                  Credentialed Accounts
ACTIVE  ACCOUNT
*       199045662654-compute@developer.gserviceaccount.com

To set the active account, run:
    $ gcloud config set account `ACCOUNT`



In [2]:
from google.cloud import storage

# Replace with your own Google Cloud Storage project and bucket name make sure the bucket name is new each time you run the notebook
PROJECT_ID = 'Your Project ID'
BUCKET_NAME = 'Your Bucket Name'

In [3]:
# Initialize the client
storage_client = storage.Client()

# Create the bucket
bucket = storage_client.create_bucket(BUCKET_NAME)

print(bucket.name)

ehc-dlarregui-4ffd8d2-productinterest5


Set up Container

In [4]:
REGION = "us-central1"
BUCKET_NAME = 'gs://'+ BUCKET_NAME
MODEL_ARTIFACT_DIR = "product-interest-model-artifacts"
REPOSITORY = "products"
IMAGE = "sklearn-image"
MODEL_DISPLAY_NAME = "product-interest-prediction"
USER_SRC_DIR="src_dir"
LOCAL_MODEL_ARTIFACTS_DIR = "model_artifacts" 
requirements_file = USER_SRC_DIR + "/requirements.txt"
predictor_file = USER_SRC_DIR + "/predictor.py"

In [5]:
%%writefile $requirements_file
fastapi
uvicorn==0.17.6
joblib~=1.0
numpy~=1.20
scikit-learn
pandas
xgboost
google-cloud-storage>=1.26.0,<2.0.0dev
google-cloud-aiplatform[prediction]>=1.43.0

Overwriting src_dir/requirements.txt


In [6]:
%%writefile $predictor_file
import os
import json
import pandas as pd
import logging
import joblib
import shutil

from google.cloud import aiplatform
from google.cloud.aiplatform.prediction.sklearn.predictor import SklearnPredictor
from google.cloud.aiplatform.utils import prediction_utils


logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)


class CprPredictor(SklearnPredictor):
    States_List = ['Washington', 'Massachusetts', 'California', 'Minnesota', 'Vermont', 'Colorado', 'Arizona']
    request_content_type = 'application/json'
    
    def __init__(self):
        return

    def load(self, artifacts_uri: str) -> None:
        super().load(artifacts_uri)
        self.label_encoder = joblib.load("labelencoder.joblib")
        # logger.info(f"Downloading artifacts from {artifacts_uri}")
        # shutil.copytree(artifacts_uri, ".", dirs_exist_ok=True)
        # logger.info("Artifacts successfully downloaded!")
        # self.label_encoder = joblib.load("labelencoder.joblib")
        # self._model = joblib.load("model.joblib")
    

    def preprocess(self, request_body: str) -> pd.DataFrame:
        inpVar = []
        if self.request_content_type == 'application/json':
            request_body = request_body
            for instance in request_body["instances"]:
                features = instance["features"]
                inpVar.append(features)
            inputs = self.transform(inpVar)
            return inputs
        else:
            raise ValueError("This model only supports application/json input")


    def transform(self, input_list: list) -> pd.DataFrame:
        X = pd.DataFrame(input_list)
        X.columns = ['club_member__c', 'campaign__c', 'state__c', 'month__c', 'case_count__c', 'case_type_return__c',
                     'case_type_shipment_damaged__c', 'pages_visited__c', 'engagement_score__c', 'tenure__c', 'clicks__c']
        X['club_member__c'] = X['club_member__c'].astype(object)
        X['month__c'] = X['month__c'].astype(object)
        X['case_type_return__c'] = X['case_type_return__c'].astype(object)
        X['case_type_shipment_damaged__c'] = X['case_type_shipment_damaged__c'].astype(object)
        X['transformed_state__c'] = X['state__c'].apply(lambda x: 'Other' if x not in self.States_List else x)
        X['transformed_cases__c'] = X['case_count__c'].apply(lambda x: 'No Cases' if x == 0 else '1 to 2 Cases' if x <= 2 else 'Greater than 2 Cases')
        X['transformed_tenure__c'] = X['tenure__c'].apply(lambda x: 'Less than 1' if x < 1 else '1 to 2 Years' if x == 1 else '1 to 2 Years' if x == 2 else '2 to 3 Years' if x == 3 else 'Greater than 3 Years')
        X = X.drop(['state__c', 'case_count__c', 'tenure__c'], axis=1)
        X = X.rename(columns={'transformed_state__c': 'state__c', 'transformed_cases__c': 'case_count__c',
                              'transformed_tenure__c': 'tenure__c'})
        return X

    def postprocess(self, prediction: pd.Series) -> dict:
        y_predicted = self.label_encoder.inverse_transform(prediction)
        return {"predictions": [{"product_interest__c": value} for value in y_predicted]}

Overwriting src_dir/predictor.py


In [7]:
# Install the same dependencies used in the serving container in the notebook
# environment.
%pip install -U --user -r $requirements_file

  Using cached fastapi-0.110.0-py3-none-any.whl (92 kB)
  Using cached starlette-0.36.3-py3-none-any.whl (71 kB)
  Using cached uvicorn-0.27.1-py3-none-any.whl (60 kB)
  Using cached fastapi-0.103.0-py3-none-any.whl (66 kB)
  Using cached fastapi-0.102.0-py3-none-any.whl (66 kB)
  Using cached fastapi-0.101.1-py3-none-any.whl (65 kB)
  Using cached fastapi-0.101.0-py3-none-any.whl (65 kB)
  Using cached fastapi-0.100.1-py3-none-any.whl (65 kB)
  Using cached fastapi-0.100.0-py3-none-any.whl (65 kB)
  Using cached fastapi-0.99.1-py3-none-any.whl (58 kB)
  Using cached pydantic-1.10.14-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
  Using cached fastapi-0.99.0-py3-none-any.whl (58 kB)
  Using cached fastapi-0.98.0-py3-none-any.whl (56 kB)
  Using cached fastapi-0.97.0-py3-none-any.whl (56 kB)
  Using cached fastapi-0.96.1-py3-none-any.whl (57 kB)
  Using cached fastapi-0.96.0-py3-none-any.whl (57 kB)
  Using cached fastapi-0.95.2-py3-none-any.whl (56 kB)
  Using cac

In [8]:
import logging

logging.basicConfig(level=logging.INFO)

In [9]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION)

import os

from google.cloud.aiplatform.prediction import LocalModel

from src_dir.predictor import CprPredictor

local_model = LocalModel.build_cpr_model(
    USER_SRC_DIR,
    f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}",
    predictor=CprPredictor,
    requirements_path=os.path.join(USER_SRC_DIR, "requirements.txt"),
    base_image="--platform=linux/amd64 alvarobartt/torch-gpu:py310-cu12.3-torch-2.2.0 AS build",

)

INFO:google.cloud.aiplatform.docker_utils.build:Running command: docker build -t us-central1-docker.pkg.dev/ehc-dlarregui-4ffd8d/products/sklearn-image --rm -f- src_dir
/opt/conda/lib/python3.10/subprocess.py:955: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stdin = io.open(p2cwrite, 'wb', bufsize)
/opt/conda/lib/python3.10/subprocess.py:961: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stdout = io.open(c2pread, 'rb', bufsize)
INFO:google.cloud.aiplatform.docker_utils.local_util:Sending build context to Docker daemon  11.82kB
INFO:google.cloud.aiplatform.docker_utils.local_util:

INFO:google.cloud.aiplatform.docker_utils.local_util:Step 1/14 : FROM --platform=linux/amd64 alvarobartt/torch-gpu:py310-cu12.3-torch-2.2.0 AS build

INFO:google.cloud.aiplatform.docker_utils.local_util:py310-cu12.3-torch-2.2.0: Pulling from alvarobartt/torch-g

In [10]:
local_model.get_serving_container_spec()

image_uri: "us-central1-docker.pkg.dev/ehc-dlarregui-4ffd8d/products/sklearn-image"
predict_route: "/predict"
health_route: "/health"

In [11]:
import json

sample =  {"instances": [{"features": [1,"Webinar","Washington",8,3,0,0,17,54,2,25]}, {"features": [0,"Year-in Review","California",1,4,0,0,16,61,1,40]}]}

with open('instances.json', 'w') as fp:
    json.dump(sample, fp)

Check the end point in docker image locally

In [12]:
with local_model.deploy_to_local_endpoint(
    artifact_uri = 'model_artifacts/', # local path to artifacts
) as local_endpoint:
    predict_response = local_endpoint.predict(
        request_file='instances.json',
        headers={"Content-Type": "application/json"},
    )

    health_check_response = local_endpoint.run_health_check()

INFO:google.cloud.aiplatform.prediction.local_endpoint:Got the project id from the global config: ehc-dlarregui-4ffd8d.


In [13]:
print(predict_response, predict_response.content)

<Response [200]> b'{"predictions": [{"product_interest__c": "Paramount Peak Convertible Pants"}, {"product_interest__c": "Paramount Peak Convertible Pants"}]}'


In [14]:
print(health_check_response, health_check_response.content)

<Response [200]> b'{}'


In [15]:
local_endpoint.print_container_logs(show_all=True)

INFO:google.cloud.aiplatform.docker_utils.run:WARNING:root:Set the number of model server workers to 4.
INFO:google.cloud.aiplatform.docker_utils.run:INFO:     Uvicorn running on http://0.0.0.0:8080 (Press CTRL+C to quit)
INFO:google.cloud.aiplatform.docker_utils.run:INFO:uvicorn.error:Uvicorn running on http://0.0.0.0:8080 (Press CTRL+C to quit)
INFO:google.cloud.aiplatform.docker_utils.run:INFO:     Started parent process [1]
INFO:google.cloud.aiplatform.docker_utils.run:INFO:uvicorn.error:Started parent process [1]
INFO:google.cloud.aiplatform.docker_utils.run:/usr/local/lib/python3.10/dist-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.4.0 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
INFO:google.cloud.aiplatform.docker_utils.run:https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
IN

Push the container to artifact registry

In [16]:
!gcloud services list

NAME                              TITLE
aiplatform.googleapis.com         Vertex AI API
artifactregistry.googleapis.com   Artifact Registry API
compute.googleapis.com            Compute Engine API
containeranalysis.googleapis.com  Container Analysis API
containerregistry.googleapis.com  Container Registry API
containerscanning.googleapis.com  Container Scanning API
notebooks.googleapis.com          Notebooks API
oslogin.googleapis.com            Cloud OS Login API
pubsub.googleapis.com             Cloud Pub/Sub API
secretmanager.googleapis.com      Secret Manager API
serviceusage.googleapis.com       Service Usage API
storage-api.googleapis.com        Google Cloud Storage JSON API


In [17]:
!gcloud services enable artifactregistry.googleapis.com

In [18]:
!gcloud artifacts repositories create {REPOSITORY} \
    --repository-format=docker \
    --location=$REGION

ERROR: (gcloud.artifacts.repositories.create) ALREADY_EXISTS: the repository already exists


In [19]:
!gcloud auth configure-docker {REGION}-docker.pkg.dev --quiet


{
  "credHelpers": {
    "gcr.io": "gcloud",
    "us.gcr.io": "gcloud",
    "eu.gcr.io": "gcloud",
    "asia.gcr.io": "gcloud",
    "staging-k8s.gcr.io": "gcloud",
    "marketplace.gcr.io": "gcloud",
    "us-central1-docker.pkg.dev": "gcloud"
  }
}
Adding credentials for: us-central1-docker.pkg.dev
gcloud credential helpers already registered correctly.


In [20]:
local_model.push_image()

/opt/conda/lib/python3.10/subprocess.py:955: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stdin = io.open(p2cwrite, 'wb', bufsize)
/opt/conda/lib/python3.10/subprocess.py:961: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stdout = io.open(c2pread, 'rb', bufsize)
INFO:google.cloud.aiplatform.docker_utils.local_util:Using default tag: latest

INFO:google.cloud.aiplatform.docker_utils.local_util:The push refers to repository [us-central1-docker.pkg.dev/ehc-dlarregui-4ffd8d/products/sklearn-image]

INFO:google.cloud.aiplatform.docker_utils.local_util:c40f7e23053c: Preparing

INFO:google.cloud.aiplatform.docker_utils.local_util:cbcb719c7f46: Preparing

INFO:google.cloud.aiplatform.docker_utils.local_util:652f89a92e8c: Preparing

INFO:google.cloud.aiplatform.docker_utils.local_util:d4dd47253b1c: Preparing

INFO:google.cloud.aiplatform.docker_u

Copy Model files from local to GCP

In [21]:
!gsutil cp model_artifacts/model.joblib {BUCKET_NAME}/{MODEL_ARTIFACT_DIR}/
!gsutil cp model_artifacts/labelencoder.joblib {BUCKET_NAME}/{MODEL_ARTIFACT_DIR}/

Copying file://model_artifacts/model.joblib [Content-Type=application/octet-stream]...
- [1 files][ 22.1 MiB/ 22.1 MiB]                                                
Operation completed over 1 objects/22.1 MiB.                                     
Copying file://model_artifacts/labelencoder.joblib [Content-Type=application/octet-stream]...
/ [1 files][  811.0 B/  811.0 B]                                                
Operation completed over 1 objects/811.0 B.                                      


Deploy to Vertex AI

Upload the custom container model

In [22]:
from google.cloud import aiplatform

In [23]:
aiplatform.init(project=PROJECT_ID, location=REGION)

Use the LocalModel instance to upload the model. It will populate the container spec automatically for you.

In [24]:
model = aiplatform.Model.upload(
    local_model=local_model,
    display_name=MODEL_DISPLAY_NAME,
    artifact_uri=f"{BUCKET_NAME}/{MODEL_ARTIFACT_DIR}",
)

INFO:google.auth.compute_engine._metadata:Compute Engine Metadata server call to universe/universe_domain returned 404, reason: <!DOCTYPE html>
<html lang=en>
  <meta charset=utf-8>
  <meta name=viewport content="initial-scale=1, minimum-scale=1, width=device-width">
  <title>Error 404 (Not Found)!!1</title>
  <style>
    *{margin:0;padding:0}html,code{font:15px/22px arial,sans-serif}html{background:#fff;color:#222;padding:15px}body{margin:7% auto 0;max-width:390px;min-height:180px;padding:30px 0 15px}* > body{background:url(//www.google.com/images/errors/robot.png) 100% 5px no-repeat;padding-right:205px}p{margin:11px 0 22px;overflow:hidden}ins{color:#777;text-decoration:none}a img{border:0}@media screen and (max-width:772px){body{background:none;margin-top:0;max-width:none;padding-right:0}}#logo{background:url(//www.google.com/images/branding/googlelogo/1x/googlelogo_color_150x54dp.png) no-repeat;margin-left:-5px}@media only screen and (min-resolution:192dpi){#logo{background:url(//ww

Set up Model Endpoint

In [25]:
endpoint = model.deploy(machine_type="n1-standard-4")

INFO:google.cloud.aiplatform.models:Creating Endpoint
INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/199045662654/locations/us-central1/endpoints/6011806324260601856/operations/6762511116623937536
INFO:google.cloud.aiplatform.models:Endpoint created. Resource name: projects/199045662654/locations/us-central1/endpoints/6011806324260601856
INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:
INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/199045662654/locations/us-central1/endpoints/6011806324260601856')
INFO:google.cloud.aiplatform.models:Deploying model to Endpoint : projects/199045662654/locations/us-central1/endpoints/6011806324260601856
INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/199045662654/locations/us-central1/endpoints/6011806324260601856/operations/3567770130957991936
INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/19904566265

In [ ]:
!gcloud auth print-access-token